# Microsoft Vision classification example
This example shows an example of a simple way to classify images from any dataset using pretrained Microsoft Vision model.

Using 1 GPU, under 10 minutes of training we can achieve score 92.92% accuracy on CIFAR-10 dataset using kNN algorithm.
We also show how to plug-in LinearClassification algorithm on top of frozen featues.

We'll start by necessary imports. We are using pytorch as model backend and sklearn LogisticRegression for simplicity and reproducibility of our results.

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
import numpy as np
from sklearn.linear_model import LogisticRegression
from progressbar import progressbar
from sklearn.neighbors import KNeighborsClassifier

And the new element: Microsoft Vision package. It gives immediate access to pretrained ResNet50 model in just 1 line. With interface very similar to torchvision

In [ ]:
import microsoftvision

Let's define how we'll preprocess images. We can notice that Microsoft Vision model is using images in BGR format, hence the swapping of image channels at the end of preprocessing

In [ ]:
class Preprocess:
    def __init__(self):
        self.preprocess = transforms.Compose([
                                           transforms.Resize(224),
                                           transforms.CenterCrop(224),
                                           transforms.ToTensor(),
                                           transforms.Normalize(mean=[0.406, 0.456, 0.485], std=[0.225, 0.224, 0.229])])

    def __call__(self, x):
        return self.preprocess(x)[[2,1,0],:,:]

Import the CIFAR-10 dataset with division to train and test sets. This can be replaced with any dataset without any changes to the rest of the code.

In [ ]:
train_dataset = CIFAR10('path', download=True, train=True, transform=Preprocess())
test_dataset = CIFAR10('path', download=True, train=False, transform=Preprocess())

Files already downloaded and verified
Files already downloaded and verified


And now, we are importing Microsoft Vision model with ResNet50 architecture. We are specifying that we want the pretrained version (same interface as torchvision).

In [ ]:
model = microsoftvision.models.resnet50(pretrained=True)

Loading Microsoft Vision pretrained model


  0%|          | 0/23 [00:00<?, ?MB/s]

Model size: 89 MB


100%|██████████| 23/23 [00:06<00:00,  6.20MB/s]


Model saved to MicrosoftVision.ResNet50.tar


Microsoft vision model is just used to extract image features, without fine-tuning. Therefore we are setting it to evaluation mode. Let's use GPU to speed-up computation.

In [ ]:
model.eval()
model.cuda()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
def get_features(dataset, model):
    all_features = []
    all_labels = []

    with torch.no_grad():
        for images, labels in progressbar(DataLoader(dataset, batch_size=128, num_workers=8)):
            images = images.cuda()
            labels = labels.cuda()
            features = model(images)

            all_features.append(features)
            all_labels.append(labels)

    return torch.cat(all_features).cpu().numpy(), torch.cat(all_labels).cpu().numpy()

We're extracting all image features from training and test set. Those features will be used to train the linear regression model and calculate accuracy score.

In [ ]:
train_features, train_labels = get_features(train_dataset, model)
test_features, test_labels = get_features(test_dataset, model)

100% (391 of 391) |######################| Elapsed Time: 0:01:40 Time:  0:01:40
100% (79 of 79) |########################| Elapsed Time: 0:00:19 Time:  0:00:19


Fit the classifier to the training set and then measure performance on test set. Whole operation will take less than 10 min using 1 GPU!

In [ ]:
# You can plug-in any classifier

#classifier = LogisticRegression(random_state=0, max_iter=1000, verbose=1, n_jobs=16)
classifier = KNeighborsClassifier(n_neighbors=5, n_jobs=16)

In [ ]:
classifier.fit(train_features, train_labels)
predictions = classifier.predict(test_features)
accuracy = np.mean((test_labels == predictions).astype(np.float)) * 100.
print(f"Accuracy: {accuracy}")

Accuracy: 92.92
